In [1]:
from google.colab import files
uploaded = files.upload()    #T1T2CommunityCoresTrusses.csv  ratings.csv

Saving RatingData.csv to RatingData.csv
Saving T1T2CommunityCoresTrusses.csv to T1T2CommunityCoresTrusses.csv


In [2]:
import pandas as pd
import numpy as np
import random
import math

In [3]:
UserCommunity= pd.read_csv("T1T2CommunityCoresTrusses.csv")   
UserCommunity.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)   #There was an unamed column in the file, needed to delete that column
UserCommunity.drop(["a"], axis=1, inplace=True)

print("Total nodes are:", UserCommunity['UserID'].count())
#UserCommunity['ComWithSubCom'] = UserCommunity['ComWithSubCom'].astype('string')
#UserCommunity['SubCommunity'] = UserCommunity['SubCommunity'].astype('string')
#UserCommunity['Community'] = UserCommunity['Community'].astype('string')
UserCommunity = UserCommunity[['UserID','ComWithSubCom']]
UserCommunity

Total nodes are: 943


,UserID,ComWithSubCom
0,1,0
1,2,10
2,3,1
3,4,1
4,5,11
...,...,...
938,939,70
939,940,20
940,941,30
941,942,71


In [4]:
CommunityLabel = 'ComWithSubCom'   #'Community'  
communityMembers = UserCommunity.groupby(CommunityLabel)['UserID'].apply(list).to_dict()
communitySizes = {key: len(value) for key, value in communityMembers.items()}
communitySizes

{0: 132,
 1: 126,
 10: 55,
 11: 50,
 20: 60,
 21: 21,
 30: 115,
 31: 22,
 40: 97,
 50: 26,
 60: 73,
 70: 57,
 71: 109}

In [5]:
MovieRatings= pd.read_csv("RatingData.csv") 
MovieRatings.drop('Timestamp', axis=1, inplace=True)
print("Total movies are:", MovieRatings['MovieID'].count())
MovieRatings.head()

Total movies are: 100000


,UserID,MovieID,Ratings
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [6]:
MergedMovieRatings = pd.merge(left=MovieRatings, right=UserCommunity, how='left', left_on='UserID', right_on='UserID') 
MergedMovieRatings.head()
print(MergedMovieRatings.loc[MergedMovieRatings['UserID'] == 5])

       UserID  MovieID  Ratings  ComWithSubCom
172         5        2        3             11
439         5       17        4             11
673         5      439        1             11
679         5      225        2             11
922         5      110        1             11
...       ...      ...      ...            ...
93172       5      419        3             11
94436       5      375        3             11
95021       5      373        3             11
96918       5      368        1             11
99848       5      174        5             11

[175 rows x 4 columns]


In [7]:
from math import sqrt
def compute_RMSE(pred, original):
    n = len(pred)
    RMSE = sqrt(sum((pred-original)**2)/n)
    return RMSE

def compute_MAE(pred, original):
    n = len(pred)
    MAE = sum(abs(original - pred))/n
    return MAE

In [8]:
FinalResult = pd.DataFrame(columns=['UserID',	'MovieID'	,'NumMoviesWatched',	'Intersection',	'Precision'	,'Recall','F-1'])

#communitySizes = {'0':113 ,'1':130, '10': 24} for testing purpose
for community in communitySizes:
  print('Processing SubCommunity: ',community)

  AllUsersInCommunity = UserCommunity[UserCommunity[CommunityLabel] == community]
  AllUsersInCommunity = AllUsersInCommunity.sample(frac = 1, random_state=1) #we use random_state to ensure the reproducibility of the results, 
                                                                                          #frac: Fraction of axis items to return
  print("Total users in community are:", AllUsersInCommunity['UserID'].count())
  #random.shuffle(AllMoviesRatedByCommunity)  #is causing keyerror
  eighty=len(AllUsersInCommunity)*0.8
  SplittingPoint=math.floor(eighty)
  #print('SplittingPoint: ',SplittingPoint)
  training_Group_Users = AllUsersInCommunity[0:SplittingPoint]['UserID'].to_list()
  testing_Group_Users = AllUsersInCommunity[SplittingPoint:(len(AllUsersInCommunity)+1)]['UserID'].to_list()

  AllMoviesRatedByCommunity = MergedMovieRatings[MergedMovieRatings[CommunityLabel] == community] 
  training_Group =AllMoviesRatedByCommunity[AllMoviesRatedByCommunity['UserID'].isin(training_Group_Users)]   
  testing_Group = AllMoviesRatedByCommunity[AllMoviesRatedByCommunity['UserID'].isin(testing_Group_Users)] 

  AvgRatingsByCommunity = training_Group.groupby('MovieID', as_index = False)['Ratings'].mean()
  AvgRatingsByTestUsers = testing_Group.groupby('MovieID', as_index = False)['Ratings'].mean()
  #print(AvgRatingsByCommunity.head())
  #print(AvgRatingsByTestUsers.head())
  df = pd.merge(left=AvgRatingsByTestUsers, right=AvgRatingsByCommunity, how='left', left_on='MovieID', right_on='MovieID') 
  df.dropna(inplace=True)
  mae = compute_MAE(df['Ratings_x'], df['Ratings_y'])
  rmse = compute_RMSE(df['Ratings_x'], df['Ratings_y'])
  print('MAE=',mae, 'RMSE = ', rmse)







Processing SubCommunity:  0
Total users in community are: 132
MAE= 0.6772538096377695 RMSE =  0.9546327120022213
Processing SubCommunity:  1
Total users in community are: 126
MAE= 0.6393290460594935 RMSE =  0.8343973003853052
Processing SubCommunity:  10
Total users in community are: 55
MAE= 0.7952460858171617 RMSE =  1.0607638274862352
Processing SubCommunity:  11
Total users in community are: 50
MAE= 0.7763540438875555 RMSE =  1.0096889612701672
Processing SubCommunity:  20
Total users in community are: 60
MAE= 0.6886313481522288 RMSE =  0.9165752610064893
Processing SubCommunity:  21
Total users in community are: 21
MAE= 1.0236945812807883 RMSE =  1.2806089876125188
Processing SubCommunity:  30
Total users in community are: 115
MAE= 0.5917003196815988 RMSE =  0.8041491634942364
Processing SubCommunity:  31
Total users in community are: 22
MAE= 0.8411083678055659 RMSE =  1.118066254490669
Processing SubCommunity:  40
Total users in community are: 97
MAE= 0.7564288567703747 RMSE =  1.

In [9]:
#create a list of max-k
num_k = [5, 10, 20, 30, 40]
#initialize lists for RMSE and MAE
RMSE = []
MAE = []

for k in num_k:   
 

SyntaxError: ignored